In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import os
import sys

proj_root = os.path.dirname(os.path.abspath("."))
# print(proj_root)
sys.path.append(proj_root)

from minatar_dqn.redo import (
    apply_redo_parametrization,
)

import torch
import torch.nn as nn


In [7]:
class SmallNetwork(nn.Module):
    def __init__(self):
        super(SmallNetwork, self).__init__()
        self.conv1 = nn.Conv2d(1, 2, kernel_size=2, stride=1, padding=0)  # 2 output channels
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(2, 2)  # Adjusted for 2 input features, 2 output features

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc1(x)
        return x

# Initialize the network
net = SmallNetwork()

with torch.no_grad():
    # For conv1, first neuron has higher mean activation than the second
    net.conv1.weight[0].fill_(-3)  # This should be affected by ReDo
    net.conv1.weight[1].fill_(3)   # This should not be affected by ReDo
    net.conv1.bias.fill_(0.1)

    # For fc1, first neuron has higher mean activation than the second
    net.fc1.weight[0].fill_(0)     # This should not be affected by ReDo
    net.fc1.weight[1].fill_(-3)    # This should be affected by ReDo
    net.fc1.bias.fill_(0.1)

# Add ReDo to the network
net = apply_redo_parametrization(net, tau=0.1, beta=1)

# Print initial weights
print("Initial Conv1 weights:", net.conv1.weight)
print("Initial FC1 weights:", net.fc1.weight)

# Dummy input - single channel 2x2 matrix
input_tensor = torch.tensor([[[[1.0, 2.0], [3.0, 4.0]]]], dtype=torch.float32)

# Forward pass before ReDo
output_before_redo = net(input_tensor)
print("Output before ReDo:", output_before_redo)

redo_scores = net.get_dormant_scores()
print("Redo Scores:", redo_scores)

print()

# Do redo resets
reset_details = net.apply_redo()
print("Reset Details:", reset_details)

# Forward pass after ReDo to see if there is any change in behavior
output_after_redo = net(input_tensor)
print("Output after ReDo:", output_after_redo)

# Weights after reset
print("Conv1 weights after ReDo:", net.conv1.weight)
print("FC1 weights after ReDo:", net.fc1.weight)

Initial Conv1 weights: Parameter containing:
tensor([[[[-3., -3.],
          [-3., -3.]]],


        [[[ 3.,  3.],
          [ 3.,  3.]]]], requires_grad=True)
Initial FC1 weights: Parameter containing:
tensor([[ 0.,  0.],
        [-3., -3.]], requires_grad=True)
Output before ReDo: tensor([[  0.1000, -90.2000]], grad_fn=<AddmmBackward0>)
Redo Scores: [tensor([0., 1.])]

Reset Details: [{'indexes': tensor([0]), 'inbound': 'conv1', 'outbound': 'fc1'}]
Output after ReDo: tensor([[  0.1000, -90.2000]], grad_fn=<AddmmBackward0>)
Conv1 weights after ReDo: Parameter containing:
tensor([[[[0.2767, 0.3235],
          [0.1546, 0.2515]]],


        [[[3.0000, 3.0000],
          [3.0000, 3.0000]]]], requires_grad=True)
FC1 weights after ReDo: Parameter containing:
tensor([[ 0.,  0.],
        [ 0., -3.]], requires_grad=True)


In [8]:
import torch
import torch.nn as nn

class SmallLinearNetwork(nn.Module):
    def __init__(self):
        super(SmallLinearNetwork, self).__init__()
        self.fc1 = nn.Linear(2, 2)  # First linear layer with 2 input and 2 output features
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(2, 2)  # Second linear layer with 2 input and 2 output features

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)  
        x = self.fc2(x)
        return x

# Initialize the network
net = SmallLinearNetwork()

# Manually set weights for the linear layers
with torch.no_grad():
    # For fc1, first neuron has higher mean activation than the second
    net.fc1.weight[0].fill_(3)   # This should not be affected by ReDo
    net.fc1.weight[1].fill_(-3)  # This should be affected by ReDo
    net.fc1.bias.fill_(0.1)

    # For fc2, first neuron has higher mean activation than the second
    net.fc2.weight[0].fill_(3)   # This should not be affected by ReDo
    net.fc2.weight[1].fill_(3)  # This should be affected by ReDo
    net.fc2.bias.fill_(0.1)


net = apply_redo_parametrization(net, tau=0.1, beta=1)

# Print initial weights
print("Initial FC1 weights:", net.fc1.weight)
print("Initial FC2 weights:", net.fc2.weight)

# Dummy input - 1x2 tensor
input_tensor = torch.tensor([[0.5, 0.5]], dtype=torch.float32)

# Forward pass before ReDo
output_before_redo = net(input_tensor)
print("Output before ReDo:", output_before_redo)

redo_scores = net.get_dormant_scores()
print("Redo Scores:", redo_scores)

print()

# Do redo resets
reset_details = net.apply_redo()
print("Reset Details:", reset_details)

# Forward pass after ReDo to see if there is any change in behavior
output_after_redo = net(input_tensor)
print("Output after ReDo:", output_after_redo)

# Weights after ReDo
print("FC1 weights after ReDo:", net.fc1.weight)
print("FC2 weights after ReDo:", net.fc2.weight)


Initial FC1 weights: Parameter containing:
tensor([[ 3.,  3.],
        [-3., -3.]], requires_grad=True)
Initial FC2 weights: Parameter containing:
tensor([[3., 3.],
        [3., 3.]], requires_grad=True)
Output before ReDo: tensor([[9.4000, 9.4000]], grad_fn=<AddmmBackward0>)
Redo Scores: [tensor([1., 0.])]

Reset Details: [{'indexes': tensor([1]), 'inbound': 'fc1', 'outbound': 'fc2'}]
Output after ReDo: tensor([[9.4000, 9.4000]], grad_fn=<AddmmBackward0>)
FC1 weights after ReDo: Parameter containing:
tensor([[ 3.0000,  3.0000],
        [ 0.6635, -0.2589]], requires_grad=True)
FC2 weights after ReDo: Parameter containing:
tensor([[3., 0.],
        [3., 0.]], requires_grad=True)
